# Final Project Data Preparation

## Imports

In [8]:
import re
import csv
import time
import spacy
import requests
import pandas as pd
import xml.etree.ElementTree as ET

## Scraping [BoardGameGeek](https://boardgamegeek.com/) Data

For this project, we selected BoardGameGeek's API as our main data source. We want to collect structured data about board games from the [BoardGameGeek XML API](https://boardgamegeek.com/wiki/page/BGG_XML_API2). This includes information like name, description, year of publication, average rating, and complexity.

We use the `/thing` endpoint with the `stats=1` parameter to include extra statistics like ratings and weight (complexity).

In [9]:
def get_game_data(game_id):
    url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&stats=1"
    response = requests.get(url)
    root = ET.fromstring(response.content)

    name = root.find(".//name").attrib["value"]
    description = root.find(".//description").text
    year = root.find(".//yearpublished").attrib["value"]
    rating = root.find(".//average").attrib["value"]
    complexity = root.find(".//averageweight").attrib["value"]

    return {
        "id": game_id,
        "name": name,
        "description": description,
        "year": int(year),
        "rating": float(rating),
        "complexity": float(complexity),
    }

In [10]:
# Example usage
game = get_game_data(174430)
print(game)

{'id': 174430, 'name': 'Gloomhaven', 'description': 'Gloomhaven  is a game of Euro-inspired tactical combat in a persistent world of shifting motives. Players will take on the roles of wandering adventurers with their own special sets of skills and their own reasons for traveling to this dark corner of the world. Players must work together out of necessity to clear out menacing dungeons and forgotten ruins. In the process, they will enhance their abilities with experience and loot, discover new locations to explore and plunder, and expand an ever-branching story fueled by the decisions they make.&#10; This is a game with a persistent and changing world that is ideally played over many game sessions. After a scenario, players will make decisions about what to do next, which will determine how the story continues, kind of like a &ldquo;Choose Your Own Adventure&rdquo; book. Playing through a scenario is a co-operative affair where players will fight against automated monsters using an in

Note: To explore all available attributes for a specific game, open this URL (replace the ID to see other games): https://boardgamegeek.com/xmlapi2/thing?id=174430&stats=1

### Creating a CSV with Multiple Games

We'll extend the previous function to handle multiple games and save them in a CSV file. This will allow us to analyze or process the dataset later.

In [11]:
def get_game_data(game_id):
    url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&stats=1"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch ID {game_id}")
        return None

    root = ET.fromstring(response.content)

    try:
        name = root.find(".//name").attrib["value"] # CAMBIAR LOS ATRIBUTOS QUE QUERAMOS
        description = root.find(".//description").text.strip()
        year = root.find(".//yearpublished").attrib["value"]
        rating = root.find(".//average").attrib["value"]
        complexity = root.find(".//averageweight").attrib["value"]

        return {
            "id": game_id,
            "name": name,
            "description": description,
            "year": int(year),
            "rating": float(rating),
            "complexity": float(complexity),
        }

    except AttributeError:
        print(f"Skipping game {game_id} (missing data)")
        return None

In [13]:
def get_top_game_ids():
    # This helper function gets the current "hot" board games (popular games based on BGG activity)
    url = "https://boardgamegeek.com/xmlapi2/hot?boardgame"
    response = requests.get(url)
    root = ET.fromstring(response.content)

    game_ids = [item.attrib["id"] for item in root.findall(".//item")]
    return game_ids

In [14]:
def save_games_to_csv(filename, game_ids):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["id", "name", "description", "year", "rating", "complexity"])
        writer.writeheader()

        for idx, game_id in enumerate(game_ids, 1):
            print(f"Fetching {idx}/{len(game_ids)}: Game ID {game_id}")
            game_data = get_game_data(game_id)

            if game_data:
                writer.writerow(game_data)

            time.sleep(1)

In [15]:
def save_top_games_to_csv(filename, num_games=100):
    game_ids = get_top_game_ids()[:num_games]  # We retrieve the top games (MAYBE CAMBIAR ESTO)

    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["id", "name", "description", "year", "rating", "complexity"])
        writer.writeheader()

        for idx, game_id in enumerate(game_ids, 1):
            print(f"Fetching {idx}/{num_games}: Game ID {game_id}")
            game_data = get_game_data(game_id)

            if game_data:
                writer.writerow(game_data)

            time.sleep(1)  # Avoid hitting API limits

In [18]:
# Save games from ID 1 to 200
game_ids = list(range(1, 201))
save_games_to_csv(filename="boardgames.csv", game_ids=game_ids)

Fetching 1/200: Game ID 1
Fetching 2/200: Game ID 2
Fetching 3/200: Game ID 3
Fetching 4/200: Game ID 4
Fetching 5/200: Game ID 5
Fetching 6/200: Game ID 6
Fetching 7/200: Game ID 7
Fetching 8/200: Game ID 8
Fetching 9/200: Game ID 9
Fetching 10/200: Game ID 10
Fetching 11/200: Game ID 11
Fetching 12/200: Game ID 12
Fetching 13/200: Game ID 13
Fetching 14/200: Game ID 14
Fetching 15/200: Game ID 15
Fetching 16/200: Game ID 16
Fetching 17/200: Game ID 17
Fetching 18/200: Game ID 18
Fetching 19/200: Game ID 19
Fetching 20/200: Game ID 20
Fetching 21/200: Game ID 21
Fetching 22/200: Game ID 22
Fetching 23/200: Game ID 23
Fetching 24/200: Game ID 24
Fetching 25/200: Game ID 25
Fetching 26/200: Game ID 26
Fetching 27/200: Game ID 27
Fetching 28/200: Game ID 28
Fetching 29/200: Game ID 29
Fetching 30/200: Game ID 30
Fetching 31/200: Game ID 31
Fetching 32/200: Game ID 32
Fetching 33/200: Game ID 33
Skipping game 33 (missing data)
Fetching 34/200: Game ID 34
Fetching 35/200: Game ID 35
Skippi

In [ ]:
# Save top 100 games (we only get 50)
# save_top_games_to_csv("top_boardgames.csv", num_games=100) # CAMBIAR NÚMERO FINAL DE GAMES

In [19]:
df = pd.read_csv("boardgames.csv")
df

,id,name,description,year,rating,complexity
0,1,Die Macher,Die Macher is a game about seven sequential po...,1986,7.58934,4.3081
1,2,Dragonmaster,Dragonmaster is a trick-taking card game based...,1981,6.65971,1.9643
2,3,Samurai,Samurai is set in medieval Japan. Players comp...,1998,7.47612,2.4423
3,4,Tal der Könige,When you see the triangular box and the luxuri...,1992,6.61540,2.6875
4,5,Acquire,"In Acquire, each player strategically invests ...",1964,7.35072,2.4923
...,...,...,...,...,...,...
114,189,Die Siedler von Nürnberg,"Based on the well-known Settlers of Catan, thi...",1999,6.90314,2.7792
115,193,Fill or Bust,Fill or Bust is all about rolling dice and pre...,1981,5.79966,1.1489
116,195,NASCAR Champions,It looks a bit like Daytona 500 with its overs...,1998,5.51500,1.1000
117,197,HellRail: Second Perdition,"Based on Dante&#039;s Inferno, HellRail: 2nd P...",1999,6.04218,2.0000


## Data Cleaning

In [20]:
# Load English spaCy model
nlp = spacy.load("en_core_web_sm")

# We implement data cleaning using spaCy
def clean_text(text):
    if pd.isna(text):
        return ""

    # Lowercase
    text = text.lower()

    # Remove HTML tags
    text = re.sub(r"<.*?>", " ", text)

    # Remove special characters and numbers
    text = re.sub(r"[^a-z\s]", " ", text)

    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # Apply spaCy NLP processing
    doc = nlp(text)

    # Remove stopwords, lemmatize words, and exclude punctuations
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

    return " ".join(tokens)

In [21]:
# Apply the cleaning function to all descriptions
df["clean_description"] = df["description"].apply(clean_text)

In [24]:
# Example: original and cleaned text for first entry
print("Original:\n", df["description"][0])
print("Cleaned:\n", df["clean_description"][0])

Original:
 Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring victory points. First, each regional election can supply one to eighty victory points, depending on the size of the region and how well your party does in it. Second, if a party wins a regional election and has some media influence in the region, then the party will receive some media-control victory points. Third, each party has a national party membership which will grow as the game progresses and this will supply a fair number of victory points. Lastly, parties score some victory points if their party platform matches the national opinions at the end of the game.&#10;&#10;The 1986 edition featured four parties from the old West Germany and

### Save clean dataset

In [27]:
df.to_csv("boardgames_clean.csv", index=False, encoding="utf-8", sep=";")